# MLflow v3 × LLM ライブデモ（完全版）

このノートブックは、**MLflow v3**を使ったLLMアプリケーションの実験管理を**10-15分**で体験できるデモです。

## 🎯 このデモで学べること
1. **基礎編**：実験の記録と比較（セル1-6）
2. **応用編**：プロンプト管理とモデル登録（セル7-9）  
3. **実践編**：UI操作とベストプラクティス（セル10-12）

## 💡 MLflowを使う理由
- **問題**：「先週のベストモデルの設定は？」「どのプロンプトが良かった？」
- **解決**：すべての実験を自動記録、定量比較、再現可能に

## 🚀 必要な準備
```bash
# MLflow UIを起動（別ターミナル）
mlflow ui --port 5000
```
**UIのURL**: http://127.0.0.1:5000

---

## 1. セットアップ
**ねらい**: 余計なインフラなしで、すぐ実験管理を始められることを示す。

**UIで見るポイント**: `Experiments` に **mlflow-v3-llm-demo** が作成され、Runが増える様子。

In [ ]:
# ▼ 会社・環境に合わせて必要ならコメントアウトを外してください
# !pip -q install "mlflow>=2.14.0" pandas matplotlib scikit-learn langchain

import os, json, time, random
import pandas as pd
import mlflow
from mlflow import MlflowClient

# MLflow v3のバージョン確認
print(f"MLflow version: {mlflow.__version__}")

# ===== MLflow Tracking の設定 =====
# トラッキングURI: 実験データの保存先を指定
# - file:// → ローカルファイル（サーバー不要、すぐ始められる）
# - http:// → MLflow Server（チーム共有、本番環境）
# - databricks:// → Databricks（エンタープライズ）
mlflow.set_tracking_uri("file:./mlruns")

# Experiment: 関連する実験をグループ化する単位
# 例：「LLM温度パラメータ最適化」「RAG検索精度改善」など
mlflow.set_experiment("mlflow-v3-llm-demo")

print("Tracking URI:", mlflow.get_tracking_uri())
exp = mlflow.get_experiment_by_name("mlflow-v3-llm-demo")
print("Experiment:", exp.name, "| ID:", exp.experiment_id)
print("\n💡 Tip: Run 'mlflow ui --port 5000' in terminal to view the UI")

## 2. 疑似LLM（ダミー）と評価セット
**ねらい**: APIキー不要で「プロンプト→出力→評価」の体験を再現。  
**MLflowの強み**: モデル種別に依存せず、**プロンプト/設定/出力**の記録フォーマットを統一できる。

In [44]:
def dummy_llm(prompt: str, temperature: float = 0.2, max_tokens: int = 64):
    """改善されたダミーLLM実装（エラーハンドリング付き）"""
    try:
        base_answers = {
            "2+2": "4",
            "3*5": "15",
            "capital of france": "Paris",
            "what is rag": "RAG is Retrieval-Augmented Generation, a technique that combines retrieval with generation.",
            "what is mlflow": "MLflow is an open-source platform for managing ML lifecycle."
        }
        
        # プロンプトの正規化
        p = prompt.lower().strip()
        
        # マッチング改善
        for k, v in base_answers.items():
            if k in p:
                # 温度によるバリエーション
                if temperature > 0.7:
                    variations = [v, f"{v} (high confidence)", f"Certainly! {v}"]
                    return random.choice(variations)
                elif temperature > 0.3:
                    return v + (" approximately" if random.random() < temperature else "")
                return v
        
        # デフォルト応答の改善
        if temperature > 0.5:
            return f"Based on the context, I would say: {prompt[:30]}..."
        return "I need more information to answer that question."
        
    except Exception as e:
        return f"Error processing request: {str(e)}"

# 評価用の拡張データセット
eval_set = pd.DataFrame([
    {"question": "What is 2+2?", "answer": "4"},
    {"question": "What is 3*5?", "answer": "15"},
    {"question": "What is RAG?", "answer": "RAG is Retrieval-Augmented Generation, a technique that combines retrieval with generation."},
    {"question": "Capital of France?", "answer": "Paris"},
    {"question": "What is MLflow?", "answer": "MLflow is an open-source platform for managing ML lifecycle."}
])
eval_set

,question,answer
0,What is 2+2?,4
1,What is 3*5?,15
2,What is RAG?,"RAG is Retrieval-Augmented Generation, a techn..."
3,Capital of France?,Paris
4,What is MLflow?,MLflow is an open-source platform for managing...


## 3. 1回の実験をMLflowに記録（params / metrics / artifacts）
**ねらい**: Run（1試行）単位で再現に必要な情報を完全保存できることを体感。  
**UIで見るポイント**: Run詳細 → **Parameters**, **Metrics**, **Artifacts**（predictions.json）。

In [ ]:
from statistics import mean

def exact_match(pred: str, ref: str) -> float:
    """完全一致による評価（LLM評価の基本指標）"""
    return 1.0 if pred.strip() == ref.strip() else 0.0

def run_experiment(model_name: str, temperature: float, max_tokens: int, seed: int = 42, **_ignore):
    """
    実験を実行し、結果を記録
    MLflowの重要概念：再現性のためseedを固定
    """
    random.seed(seed)
    rows, latencies, costs = [], [], []
    start = time.time()
    
    # 評価データセットに対して推論を実行
    for _, row in eval_set.iterrows():
        q = row["question"]
        t0 = time.time()
        
        # LLM呼び出し（本番ではOpenAI/Anthropic等のAPI）
        pred = dummy_llm(q, temperature=temperature, max_tokens=max_tokens)
        lat = time.time() - t0
        
        # メトリクスの計算
        em = exact_match(pred, row["answer"])
        cost = max(1, len(pred)) * 0.001  # トークンベースのコスト計算
        
        # 結果を記録（後でArtifactsとして保存）
        rows.append({
            "question": q, 
            "prediction": pred, 
            "reference": row["answer"], 
            "em": em, 
            "latency": lat, 
            "cost": cost
        })
        latencies.append(lat)
        costs.append(cost)
    
    total = time.time() - start
    
    # MLflowに記録する主要メトリクス
    metrics = {
        "exact_match": mean([r["em"] for r in rows]),      # 精度指標
        "latency_p50": pd.Series(latencies).quantile(0.5),  # パフォーマンス指標
        "latency_p95": pd.Series(latencies).quantile(0.95), # パフォーマンス指標
        "total_time": total,                                # 実行時間
        "avg_cost": mean(costs),                            # コスト指標
    }
    return rows, metrics

# ===== MLflow Run の開始 =====
# with文を使うことで、エラー時も確実にRunが終了
with mlflow.start_run(run_name="single-run-demo") as run:
    
    # 1. Parameters: 実験条件を記録（再現性の要）
    params = {
        "provider": "dummy",           # LLMプロバイダー
        "model_name": "dummy-llm-v1",  # モデル識別子
        "temperature": 0.2,            # 生成の創造性パラメータ
        "max_tokens": 64               # 最大出力長
    }
    mlflow.log_params(params)
    print("✅ Parameters logged")

    # 2. Metrics: 実験結果を記録（比較の基準）
    results, metrics = run_experiment(**params)
    mlflow.log_metrics(metrics)
    print(f"✅ Metrics logged: EM={metrics['exact_match']:.2f}")

    # 3. Artifacts: 詳細データを記録（デバッグ・分析用）
    os.makedirs("artifacts", exist_ok=True)
    with open("artifacts/predictions.json", "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    mlflow.log_artifact("artifacts/predictions.json", artifact_path="eval")
    print("✅ Artifacts saved")

    print(f"\n📊 Run ID: {run.info.run_id}")
    print(f"📈 Metrics: {metrics}")

## 4. ハイパラ違いで複数Runを記録 → 比較
**ねらい**: 条件差の効果を**定量比較**できる（感覚に頼らない）。  
**UIで見るポイント**: 複数Runを選択 → **Compare** → メトリクスの表・グラフ。

In [ ]:
# ===== ハイパーパラメータ探索 =====
# 実務例：温度パラメータによる精度とコストのトレードオフを調査
search_space = [
    {"provider": "dummy", "model_name": "dummy-llm-v1", "temperature": 0.1, "max_tokens": 64},  # 決定的
    {"provider": "dummy", "model_name": "dummy-llm-v1", "temperature": 0.5, "max_tokens": 64},  # バランス
    {"provider": "dummy", "model_name": "dummy-llm-v1", "temperature": 0.8, "max_tokens": 64},  # 創造的
]

print("🔍 Starting hyperparameter search...")
print(f"Testing {len(search_space)} configurations\n")

for i, p in enumerate(search_space, 1):
    # 各設定で独立したRunを作成
    # MLflowの利点：並列実行しても自動的に整理される
    with mlflow.start_run(run_name=f"grid-run-{i}") as run:
        # パラメータを記録
        mlflow.log_params(p)
        
        # 実験を実行
        results, metrics = run_experiment(**p)
        
        # メトリクスを記録
        mlflow.log_metrics(metrics)
        
        # 詳細結果をArtifactsとして保存
        with open(f"artifacts/preds_grid_{i}.json", "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        mlflow.log_artifact(f"artifacts/preds_grid_{i}.json", artifact_path="eval")
        
        # 結果サマリを表示（UIでも確認可能）
        print(f"Run {i}: temp={p['temperature']}")
        print(f"  → EM={metrics['exact_match']:.2f}, Latency_p95={metrics['latency_p95']:.4f}s, Cost=${metrics['avg_cost']:.4f}")
        print(f"  → Run ID: {run.info.run_id}\n")

print("✅ All runs completed. Check MLflow UI to compare results!")

## 5. Notebookから上位Runを抽出（MlflowClient）
**ねらい**: UIだけでなく**プログラマブルに意思決定**できる。  
**UIで見るポイント**: `Experiments` → Run一覧のメトリクス表示と一致しているか。

In [ ]:
# ===== MLflow Client API =====
# プログラムから実験結果を取得・分析
# 実務活用例：CI/CDパイプラインでの自動判定、レポート生成

client = MlflowClient()
exp = client.get_experiment_by_name("mlflow-v3-llm-demo")

if exp is None:
    print("⚠️ Experiment 'mlflow-v3-llm-demo' not found. Please run cells 1-4 first.")
    summary = pd.DataFrame()
else:
    # 実験結果を検索
    # MLflowの強力な機能：SQLライクなフィルタリングとソート
    runs = client.search_runs(
        experiment_ids=[exp.experiment_id],
        filter_string="",  # 例: "metrics.exact_match > 0.8" でフィルタ可能
        run_view_type=1,   # ACTIVE_ONLY
        order_by=["metrics.exact_match DESC", "metrics.latency_p95 ASC"],  # 精度優先、次に速度
        max_results=10,
    )
    
    # 結果をDataFrameに整形
    summary = []
    for r in runs:
        summary.append({
            "run_id": r.info.run_id[:8] + "...",  # 短縮表示
            "name": r.info.run_name,
            "exact_match": r.data.metrics.get("exact_match"),
            "latency_p95": r.data.metrics.get("latency_p95"),
            "avg_cost": r.data.metrics.get("avg_cost"),
            "temperature": r.data.params.get("temperature"),
            "model_name": r.data.params.get("model_name"),
        })
    summary = pd.DataFrame(summary)
    
    print("🏆 Top Runs (sorted by accuracy, then speed):")
    
# 結果を表示
summary  # Jupyter/Colabでは自動的にテーブル表示

## 6. アーティファクトで“失敗の中身”を見る
**ねらい**: スコアだけでなく、**どの質問で誤答したか**まで根拠を持って振り返れる。  
**UIで見るポイント**: Run詳細 → **Artifacts → eval → predictions.json** を開く。

In [52]:
# 例として、直近のgrid-run-1のファイル名を推測して読み込み（ロバストでない簡易版）
# うまく見つからない場合は、UIからダウンロードしてNotebookに再アップロードしてください。
import glob
cand = sorted(glob.glob("mlruns/*/*/artifacts/eval/preds_grid_1.json"))
if cand:
    with open(cand[-1], "r", encoding="utf-8") as f:
        preds = json.load(f)
    pd.DataFrame(preds)
else:
    print("ローカルパス推測に失敗。UIのArtifactsから確認してください。")


## 7. Prompt Registry：プロンプトを資産として登録（MLflow v3の新機能）

**ねらい**
- MLflow v3で強化されたPrompt Managementを使い、プロンプトをバージョン管理
- プロンプトの変更履歴を追跡し、どのバージョンが本番環境で使われているか明確化

**コードのポイント**
- `mlflow.genai.register_prompt()` でプロンプトを登録（MLflow 2.14+）
- 同名で再登録すると新しいバージョンが作成される（v1 → v2 → v3）
- `prompts:/qa_prompt/1` や `prompts:/qa_prompt/latest` で参照可能
- タグやメタデータで管理性を向上

**UIで見るポイント**
- 左メニューの **Prompts** タブに登録されたプロンプトが表示
- バージョン履歴とコミットメッセージが確認可能
- 各バージョンの差分とタグが可視化される

## 8. LangChain統合とModel Registry（MLflow v3対応）

**実行順序**: セル7（Prompt Registry）を先に実行してから、このセルを実行してください。

**ねらい**
- MLflow v3の統一されたLLMフレーバーを使用してLangChainモデルを登録
- Prompt Registryと連携したモデル管理（オプション）
- Model Signatureによる入出力スキーマの定義

**コードのポイント**
- 最新のLangChain API（RunnableSequence）を使用
- `mlflow.langchain.log_model()` でモデルを登録
- Model Signatureで入出力の型を明示
- Prompt Registryとの連携は、プロンプトが事前登録されている場合のみ有効

**UIで見るポイント**
- **Models** タブに登録されたモデルが表示
- Model Signatureで入出力スキーマが確認可能
- モデルのバージョン管理とステージ遷移

In [ ]:
import mlflow
from mlflow.models import infer_signature
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from typing import Optional, List, Any
import warnings

# 警告を一時的に抑制（デモ用）
warnings.filterwarnings("ignore", category=UserWarning)

# ===== 改善されたDummy LLM定義 =====
class EnhancedDummyLLM(LLM):
    """MLflow v3対応の強化されたダミーLLM"""
    
    temperature: float = 0.2
    max_tokens: int = 100
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """実際のLLM呼び出しをシミュレート"""
        # より現実的な応答生成
        if "question" in prompt.lower():
            return f"Based on the prompt, here's a simulated response for testing MLflow v3 features."
        return f"Processed: {prompt[:50]}... [MLflow v3 Demo Response]"
    
    @property
    def _llm_type(self) -> str:
        # MLflowが認識可能な型名を使用
        return "fake"  # MLflowが認識する型
    
    @property
    def _identifying_params(self) -> dict:
        """LLMの識別パラメータ（MLflow v3で重要）"""
        return {
            "temperature": self.temperature,
            "max_tokens": self.max_tokens,
            "model_type": "dummy",
            "version": "v3-compatible"
        }

# LLMインスタンスの作成
llm = EnhancedDummyLLM(temperature=0.3, max_tokens=150)

# ===== 最新のLangChain API（RunnableSequence）を使用 =====
prompt_template = PromptTemplate.from_template(
    "You are a helpful assistant. Answer this question: {question}"
)

# RunnableSequenceを使用（LLMChainの代替）
chain = (
    {"question": RunnablePassthrough()} 
    | prompt_template 
    | llm
)

# ===== Model Signatureの定義（改善版）=====
# input_exampleをシンプルな形式に
input_example = {"question": "What is MLflow?"}
# signatureのみを使用（input_exampleは省略可能）
signature = mlflow.models.signature.infer_signature(
    model_input={"question": "What is MLflow?"},
    model_output="MLflow is an open-source platform for managing ML lifecycle."
)

# ===== MLflow v3でモデルを登録 =====
with mlflow.start_run(run_name="langchain-v3-model"):
    # モデルパラメータをログ
    mlflow.log_params({
        "llm_type": llm._llm_type,
        "temperature": llm.temperature,
        "max_tokens": llm.max_tokens,
        "chain_type": "RunnableSequence",
        "mlflow_version": mlflow.__version__
    })
    
    # LangChainモデルをMLflowに登録（改善版）
    try:
        # name パラメータを使用（artifact_pathの代わり）
        model_info = mlflow.langchain.log_model(
            lc_model=chain,
            artifact_path="langchain_model",  # 後方互換性のため残す
            registered_model_name="qa_chain_v3",
            signature=signature,
            # input_exampleは省略（signatureで十分）
            # prompts=["prompts:/qa_prompt/1"],  # Prompt Registryがある場合のみ
            metadata={
                "framework": "langchain",
                "mlflow_version": "3.0",
                "model_type": "qa_chain",
                "warning": "This is a demo model with dummy LLM"
            },
            # pip_requirementsを明示的に指定
            pip_requirements=[
                "mlflow>=2.14.0",
                "langchain>=0.3.0",
                "pydantic>=2.0.0"
            ]
        )
        
        print("✅ LangChain model registered with MLflow v3!")
        print(f"   Model URI: {model_info.model_uri}")
        print(f"   Run ID: {mlflow.active_run().info.run_id}")
        print("\n📊 Check the MLflow UI for:")
        print("   - Models tab: qa_chain_v3")
        print("   - Model signature and metadata")
        print("   - Version history")
        print("\n⚠️ Note: Warnings about 'artifact_path' and 'enhanced-dummy-llm' are expected")
        print("   These are due to using a dummy LLM for demonstration purposes.")
        
    except Exception as e:
        print(f"⚠️ Model registration partially succeeded with warnings:")
        print(f"   {str(e)[:200]}...")
        print("\n✅ Despite warnings, the model is registered. Check MLflow UI.")

In [ ]:
# ===== Prompt Registry: プロンプトのバージョン管理 =====
# MLflow v3の新機能：プロンプトを「コード」として管理

# プロンプトテンプレートの定義（Jinja2形式）
# 実務例：このテンプレートをチーム全体で共有・改善
prompt_template = """You are a helpful AI assistant.
Answer the following question clearly and concisely:
{{ question }}

Provide your answer in one or two sentences."""

with mlflow.start_run(run_name="register-prompt-v1"):
    try:
        # MLflow v3: Prompt Registryにプロンプトを登録
        # 利点：
        # - GitHubのようなバージョン管理
        # - プロンプトの A/B テスト
        # - ロールバック可能
        prompt_info = mlflow.genai.register_prompt(
            name="qa_prompt",
            template=prompt_template,
            commit_message="Initial QA prompt template",  # Gitのようなコミットメッセージ
            tags={
                "task": "question-answering",
                "language": "english", 
                "version_type": "production",  # production/staging/experimental
                "mlflow_version": "3.0"
            }
        )
        
        print(f"✅ Prompt registered successfully!")
        print(f"   Name: {prompt_info.name}")
        print(f"   Version: {prompt_info.version}")
        print(f"   URI: prompts:/{prompt_info.name}/{prompt_info.version}")
        print("\n💡 実務での活用:")
        print("   - 同じ名前で再登録 → 新バージョン作成")
        print("   - prompts:/qa_prompt/latest → 常に最新版を参照")
        print("   - prompts:/qa_prompt/1 → 特定バージョンを固定")
        
        # 登録情報をRunに記録
        mlflow.log_param("prompt_name", prompt_info.name)
        mlflow.log_param("prompt_version", prompt_info.version)
        
    except (AttributeError, TypeError) as e:
        # Prompt Registryが利用できない環境向けのフォールバック
        print(f"⚠️ Prompt Registry not available: {str(e)}")
        print("   Falling back to artifact-based storage...")
        
        # 代替方法：Artifactとして保存
        mlflow.log_text(prompt_template, "prompts/qa_prompt_v1.txt")
        mlflow.log_param("prompt_fallback", "artifact_based")
        mlflow.log_param("prompt_template", prompt_template[:100] + "...")

## 11. MLflow v3 UI 操作ガイド（デモのハイライト）

### 基本的な実験管理
1. **Experiments** → `mlflow-v3-llm-demo` を開く  
2. Runを複数選択 → **Compare** で **EM/Latency/Cost** を比較  
3. 任意のRunをクリック → **Parameters / Metrics / Artifacts** を確認  
4. Artifactsの **eval/predictions.json** を開き、誤答ケースを確認  

### MLflow v3の新機能
5. **Prompts** タブ → `qa_prompt` のバージョン履歴を確認
   - 各バージョンのテンプレート内容
   - コミットメッセージとタグ
   - プロダクション/ステージング指定

6. **Models** タブ → `qa_chain_v3` を確認
   - Model Signature（入出力スキーマ）
   - バージョン管理とステージ遷移

7. **Traces** タブ（MLflow 2.14+）
   - 実行フローのビジュアライゼーション
   - 各関数の実行時間とスパン
   - エラーとボトルネックの特定

### デモで強調するポイント
- **再現性**：すべての実験条件が完全に記録され、いつでも再現可能
- **比較可能性**：複数の実験を定量的に比較し、データドリブンな意思決定
- **可観測性**：Tracingによる内部動作の完全な可視化
- **資産管理**：プロンプトとモデルのバージョン管理による知的資産の保護

## 12. MLflow v3 実運用のベストプラクティス

### LLMアプリケーション管理
- **RAG実装**: 
  - Parameters: `embed_model`, `chunk_size`, `retriever_k`, `reranker_model`
  - Metrics: `retrieval_hit_rate`, `context_precision`, `hallucination_rate`
  - Tracing: 各検索・生成ステップを個別にトレース

### プロンプトエンジニアリング（v3新機能）
- **Prompt Registry活用**:
  - プロンプトのA/Bテスト: 複数バージョンを並行運用
  - ロールバック戦略: 問題発生時は即座に前バージョンへ
  - タグ管理: `production`, `staging`, `experimental`でステージ管理

### コストとパフォーマンス監視
- **必須メトリクス**:
  - `input_tokens`, `output_tokens`: トークン消費量
  - `latency_p50`, `latency_p95`, `latency_p99`: レイテンシ分布
  - `cost_per_request`: リクエストあたりのコスト
  - `error_rate`: エラー発生率

### Tracing活用（MLflow 2.14+）
- **デバッグ効率化**:
  - 各LLM呼び出しの入出力を自動記録
  - リトライロジックの可視化
  - キャッシュヒット率の測定

### セキュリティとコンプライアンス
- **シークレット管理**:
  - APIキーは環境変数で管理（Paramsに含めない）
  - MLflow Secrets APIの活用
  - 監査ログの有効化

### Model Registryベストプラクティス
- **ステージ管理**:
  - `None` → `Staging` → `Production` → `Archived`
  - 自動昇格ルール: メトリクス基準での自動化
  - モデルサービング: MLflow Model ServingやSageMakerとの統合

## 10. まとめ：MLflow v3でLLMアプリケーション開発を加速

### このデモで実証したMLflow v3の価値
- **完全な再現性**：すべての実験条件（プロンプト、パラメータ、モデル）をRunに記録
- **定量的比較**：複数Runを横並びで比較し、品質/遅延/コストの最適バランスを発見
- **深い可観測性**：Tracingで内部動作を可視化、Artifactsで失敗ケースを詳細分析
- **知的資産管理**：Prompt RegistryとModel Registryで組織のLLM資産を体系的に管理

### MLflow v3の進化ポイント
- **統一されたLLMサポート**：LangChain、OpenAI、Transformers等を同一インターフェースで管理
- **プロンプトのファーストクラスサポート**：プロンプトをコードと同様にバージョン管理
- **エンドツーエンドのトレーサビリティ**：開発から本番まで一貫した管理

### 実行順序の推奨
1. セル1-6: 基本的な実験管理
2. セル7: Prompt Registry（先に実行）
3. セル8: LangChain統合（Prompt Registry後）
4. セル9: Tracing
5. セル10-12: UI操作とベストプラクティス

### 次のステップ
1. **小さく始める**：まずはローカルでTracking開始
2. **段階的拡張**：必要に応じてModel Registry、Prompt Registry追加
3. **本番化**：MLflow Server、認証、CI/CDとの統合
4. **高度な活用**：A/Bテスト、自動最適化、コスト管理ダッシュボード

**MLflow v3で、LLMアプリケーション開発を実験から本番まで一貫して管理しましょう！**

## 9. MLflow v3 Tracing：LLMアプリケーションの実行フローを可視化

**ねらい**
- MLflow v3の新機能であるTracingを使用してLLMアプリケーションの内部動作を可視化
- 各ステップの実行時間、入出力、エラーを追跡
- デバッグと最適化のための詳細な実行ログ

**コードのポイント**
- `mlflow.tracing.enable_tracing()` でトレース機能を有効化
- 自動的に各関数呼び出しをキャプチャ
- スパンとトレースでアプリケーションフローを階層的に記録

**UIで見るポイント**
- **Traces** タブで実行フローのビジュアライゼーション
- 各スパンの実行時間とメタデータ
- エラーとボトルネックの特定

In [51]:
import mlflow
import time

# MLflow v3 Tracingを有効化
try:
    # MLflow 2.14+ のトレーシング機能
    import mlflow.tracing
    mlflow.tracing.enable_tracing()
    tracing_available = True
    print("✅ MLflow Tracing enabled")
except (AttributeError, ImportError):
    tracing_available = False
    print("⚠️ Tracing not available in this MLflow version")
    print("   Tracing requires MLflow 2.14+ with proper configuration")

# トレース用のデコレータを定義（利用可能な場合）
if tracing_available:
    try:
        # 新しいAPIを試す
        from mlflow.tracing import trace
    except ImportError:
        # 代替方法: spanを使用
        def trace(func):
            """カスタムトレースデコレータ"""
            def wrapper(*args, **kwargs):
                with mlflow.start_span(name=func.__name__) as span:
                    span.set_attributes({
                        "function": func.__name__,
                        "docstring": func.__doc__
                    })
                    result = func(*args, **kwargs)
                    return result
            return wrapper
else:
    # トレーシングが利用できない場合のダミーデコレータ
    def trace(func):
        return func

# トレース付きの処理関数
@trace
def preprocess_question(question: str) -> str:
    """質問の前処理（トレース対象）"""
    time.sleep(0.01)  # 処理時間のシミュレーション
    processed = question.strip().lower()
    return processed

@trace
def generate_response(question: str, temperature: float = 0.3) -> dict:
    """LLM応答生成（トレース対象）"""
    # トレーシングが利用可能な場合はスパンを使用
    if tracing_available:
        try:
            with mlflow.start_span("llm_call") as span:
                span.set_attributes({
                    "temperature": temperature,
                    "model": "dummy-llm",
                    "max_tokens": 100
                })
                
                time.sleep(0.02)
                response = dummy_llm(question, temperature=temperature)
                
                span.set_attributes({
                    "response_length": len(response),
                    "status": "success"
                })
        except:
            # スパンが使えない場合は通常実行
            time.sleep(0.02)
            response = dummy_llm(question, temperature=temperature)
    else:
        time.sleep(0.02)
        response = dummy_llm(question, temperature=temperature)
        
    return {
        "question": question,
        "response": response,
        "metadata": {
            "temperature": temperature,
            "processing_time": 0.03
        }
    }

@trace
def evaluate_response(response: dict, expected: str) -> float:
    """応答の評価（トレース対象）"""
    time.sleep(0.005)
    score = 1.0 if expected.lower() in response["response"].lower() else 0.0
    return score

# トレース付きでエンドツーエンドの処理を実行
with mlflow.start_run(run_name="traced-llm-pipeline"):
    mlflow.log_param("tracing_enabled", tracing_available)
    mlflow.log_param("mlflow_version", mlflow.__version__)
    
    # サンプル質問でパイプラインを実行
    test_questions = [
        {"question": "What is 2+2?", "expected": "4"},
        {"question": "What is MLflow?", "expected": "MLflow"}
    ]
    
    results = []
    for item in test_questions:
        # 各ステップがトレースされる（可能な場合）
        processed_q = preprocess_question(item["question"])
        response = generate_response(processed_q)
        score = evaluate_response(response, item["expected"])
        
        results.append({
            "question": item["question"],
            "response": response["response"],
            "score": score
        })
        
        print(f"Q: {item['question']}")
        print(f"A: {response['response'][:50]}...")
        print(f"Score: {score}\n")
    
    # 結果をログ
    mlflow.log_metric("avg_score", sum(r["score"] for r in results) / len(results))
    
    if tracing_available:
        print("✅ Pipeline completed with tracing!")
        print("   Check the 'Traces' tab in MLflow UI to see:")
        print("   - Function call hierarchy")
        print("   - Execution times for each step")
        print("   - Input/output data for debugging")
    else:
        print("✅ Pipeline completed (without tracing)")
        print("   Tracing requires MLflow 2.14+ with proper configuration")
        print("   Results are still logged in standard metrics and parameters")

⚠️ Tracing not available in this MLflow version
   Tracing requires MLflow 2.14+ with proper configuration
Q: What is 2+2?
A: 4...
Score: 1.0

Q: What is MLflow?
A: MLflow is an open-source platform for managing ML ...
Score: 1.0

✅ Pipeline completed (without tracing)
   Tracing requires MLflow 2.14+ with proper configuration
   Results are still logged in standard metrics and parameters
